In [417]:
class Piece():
    location_occupied = {}
    pressured_by_white = {}
    pressured_by_black = {}
    
    def __init__(self, name, location, is_white, single_move):
        self.name = name
        self.location = location
        self.location_occupied[name] = location
        self.is_white = is_white
        self.single_move = single_move
        self.movement = []
        self.pinned = []
        self.directions = self.setDirections()
        
    def setDirections(self, directions=False):
        result = []
        if directions:
            for direction in directions:
                adjusting_directions = direction.split("-")
                result.append(self.locationAdjuster(adjusting_directions))            
        else:
            for direction in self.movement_directions:
                adjusting_directions = direction.split("-")
                result.append(self.locationAdjuster(adjusting_directions))
        return result
    def setPressuredLists(self):
        for x in range(0,64):
            self.pressured_by_white[(f"{x//8, x%8}")] = 0
            self.pressured_by_black[(f"{x//8, x%8}")] = 0
    def showName(self):
        print(self.name)
    def showLocation(self):
        print(self.location)
    def showOccupiedLocations(self):
        print(self.location_occupied)
    def locationColor(self):
        return self.is_white
    def showMovement(self):
        print(self.movement)
    def showPinned(self):
        print(self.pinned)
    def showMovementDirections(self):
        print(self.movement_directions)
     
    def dictNameFromLocation(self, location):
        for key in self.location_occupied:
            if self.location_occupied[key] == location:
                return key
    
    def locationAdjuster(self, directions):
        row_adjustment = 0
        column_adjustment = 0
        for direction in directions:
            if direction == "north":
                row_adjustment -= 1
            if direction == "south":
                row_adjustment += 1
            if direction == "west":
                column_adjustment -= 1
            if direction == "east":
                column_adjustment += 1
        return [row_adjustment, column_adjustment]   
    def locationChecker(self, row_adjustment, column_adjustment):
        result = []
        counter = 0
        looping = True
        adjusted_row = self.location[0] + row_adjustment
        adjusted_column = self.location[1] + column_adjustment
        while looping:   
            if adjusted_row < 0 or adjusted_row > 7:
                break
            if adjusted_column < 0 or adjusted_column > 7:
                break
            
            for value in self.location_occupied:
                if self.location_occupied[value] == [adjusted_row, adjusted_column]:
                    is_white = chess_pieces[value].locationColor()
                    if is_white:
                        counter += 2
                    else:
                        counter += 1
            
            if counter > 2:
                counter = 2
                        
            result.append(counter)
            adjusted_row += row_adjustment
            adjusted_column += column_adjustment

            if self.single_move:
                break
        return result
    def locationName(self, row_adjustment, column_adjustment, single_move):
        encounters = {}
        looping = True
        adjusted_row = self.location[0] + row_adjustment
        adjusted_column = self.location[1] + column_adjustment
        while looping:   
            if adjusted_row < 0 or adjusted_row > 7:
                break
            if adjusted_column < 0 or adjusted_column > 7:
                break
        
            name = self.dictNameFromLocation([adjusted_row, adjusted_column])
            if name:
                encounters[name] = [adjusted_row, adjusted_column]
                
            adjusted_row += row_adjustment
            adjusted_column += column_adjustment
            
            if single_move:
                break
        return encounters
    
    def setMoves(self):
        self.movement = []
        self.pinned = []
        for direction in self.directions:
            self.movement.append(self.locationChecker(direction[0], direction[1]))
            
        for x in range(len(self.movement)):
            previous_counter = 0
            self.pinned.append([])
            for y in range(len(self.movement[x])):
                if self.movement[x][y] < 2 or previous_counter < 2:
                    self.pinned[x].append(1)
                previous_counter = self.movement[x][y]
        self.addPinnedMoves()
    def addPinnedMoves(self):
        for movement_direction in range(len(self.pinned)):
            if self.pinned[movement_direction] != None:
                adjusted_row = self.location[0] + self.directions[movement_direction][0]
                adjusted_column = self.location[1] + self.directions[movement_direction][1]
                for move in self.pinned[movement_direction]:
                    if self.is_white:
                        self.pressured_by_white[(f"{adjusted_row, adjusted_column}")] += 1
                    else:
                        self.pressured_by_black[(f"{adjusted_row, adjusted_column}")] += 1
                    adjusted_row += self.directions[movement_direction][0]
                    adjusted_column += self.directions[movement_direction][1] 
    def removePinnedMoves(self):
        for movement_direction in range(len(self.pinned)):
            if self.pinned[movement_direction] != None:
                adjusted_row = self.location[0] + self.directions[movement_direction][0]
                adjusted_column = self.location[1] + self.directions[movement_direction][1]
                for move in self.pinned[movement_direction]:
                    if self.is_white:
                        self.pressured_by_white[(f"{adjusted_row, adjusted_column}")] -= 1
                    else:
                        self.pressured_by_black[(f"{adjusted_row, adjusted_column}")] -= 1
                    adjusted_row += self.directions[movement_direction][0]
                    adjusted_column += self.directions[movement_direction][1] 
        self.pinned = []            
    
    def movePiece(self, new_location):
        self.removePinnedMoves()
        piece_on_new_location = self.dictNameFromLocation(new_location)
        if piece_on_new_location:
            chess_pieces[piece_on_new_location].removePinnedMoves()
            chess_pieces.pop(piece_on_new_location)
            self.location_occupied.pop(piece_on_new_location)
        
        encounters_old_location = self.updateLocation()
        self.location = new_location
        self.location_occupied[self.name] = new_location
        self.setMoves()
        encounters_new_location = self.updateLocation()
        encounters = {**encounters_old_location, **encounters_new_location}
        
        for encounter in encounters:  
            chess_pieces[encounter].removePinnedMoves()
            chess_pieces[encounter].setMoves()        
            
        
    def updateLocation(self):
        update_movement = []
        encounters= {}
        update_directions = ["north", "north-east", "east", "south-east", "south", "south-west", "west", "north-west",
                            "north-north-east", "east-east-north", "east-east-south", "south-south-east", 
                            "south-south-west", "west-west-south", "west-west-north", "north-north-west"]
        directions = self.setDirections(update_directions)
        for direction in directions:
            update_movement.append([direction[0], direction[1]])
        
        for direction in update_movement:
            if direction[0] == 2 or direction[0] == -2 or direction[1] == 2 or direction[1] == -2:
                lst = self.locationName(direction[0], direction[1], True)
                for encounter in lst:
                    encounters[encounter] = lst[encounter]
            else:
                lst = self.locationName(direction[0], direction[1], False)
                for encounter in lst:
                    encounters[encounter] = lst[encounter]
        return encounters
class Rook(Piece):
    movement_directions = ["north", "east", "south", "west"]
class Bischop(Piece):
    movement_directions = ["north-east", "south-east", "south-west", "north-west"]    
class King(Piece):
    movement_directions = ["north", "north-east", "east", "south-east", "south", "south-west", "west", "north-west"]
class Queen(Piece):
    movement_directions = ["north", "north-east", "east", "south-east", "south", "south-west", "west", "north-west"]
class Knight(Piece):
    movement_directions = ["north-north-east", "east-east-north", "east-east-south", "south-south-east", 
                           "south-south-west", "west-west-south", "west-west-north", "north-north-west"]
class Pawn(Piece):
    ## directions for pawns are vertical, enpassant east, enpassant west, dubble vertical.
    ## the vertical movement for white is south and for black it is north
    movement_directions = []
    
    def __init__(self, name, location, is_white, single_move):
        super().__init__(name, location, is_white, single_move)
        if self.is_white:
            self.movement_directions = ["south", "south-east", "south-west", "south-south"]
        else:
            self.movement_directions = ["north", "north-east", "north-west", "north-north"]
        self.directions = self.setDirections()
    
    def setMoves(self):
        self.movement = []
        self.pinned = []
        for x in range(len(self.directions)):
            self.pinned.append([])
            location_score = self.locationChecker(self.directions[x][0], self.directions[x][1])
            if location_score:
                if x == 0:
                    if location_score == 0:
                        self.movement.append(0)
                    else:
                        self.movement.append(2)
                elif x == 3:
                    if (location_score[0] + self.movement[0]) == 0:
                        self.movement.append(0)
                    else:
                        self.movement.append(2)
                else:
                    if location_score == 1:
                        self.movement.append(1)
                    else:
                        self.movement.append(2)
                    self.pinned[x].append(1)
        self.addPinnedMoves()
    
    def addPinnedMoves(self):
        for x in range(len(self.pinned)):
            if x == 1 or x == 2:
                for y in range(len(self.pinned[x])):
                    if y:
                        if self.is_white:
                            self.pressured_by_white[(f"{(self.location[0]+1), (self.location[1]+self.directions[x][1])}")] += 1
                        else:
                            self.pressured_by_black[(f"{(self.location[0]-1), (self.location[1]+self.directions[x][1])}")] += 1
        
    def removePinnedMoves(self):
        for x in range(len(self.pinned)):
            if x == 1 or x == 2:
                for y in range(len(self.pinned[x])):
                    if y:
                        if self.is_white:
                            self.pressured_by_white[(f"{(self.location[0]+1), (self.location[1]+self.directions[x][1])}")] -= 1
                        else:
                            self.pressured_by_black[(f"{(self.location[0]-1), (self.location[1]+self.directions[x][1])}")] -= 1


In [418]:
import pandas as pd
from curses.ascii import isupper

df = pd.DataFrame(index=range(8), columns=range(8))
df_white = pd.DataFrame(index=range(8), columns=range(8))
df_black = pd.DataFrame(index=range(8), columns=range(8))

start_fen = "RNBQKBNR/PPPPPPPP/8/8/8/8/pppppppp/rnbqkbnr"
def createBoard():
    class_dictionary = {"R": Rook, "N": Knight, "B": Bischop, "Q": Queen, "K":King, "P": Pawn}
    class_single_move = {"R": False, "N": True, "B": False, "Q": False, "K":True, "P": True}
    class_spelled_out = {"R": "Rook", "N": "Knight", "B": "Bischop", "Q": "Queen", "K":"King", "P": "Pawn"}
    chess_pieces = {}
    start_fen = "RNBQKBNR/PPPPPPPP/8/8/8/8/pppppppp/rnbqkbnr"
    numberString = "123456789"

    ## there must be a better way for this
    chess_piece_counter = {"R": 0, "r": 0, "B": 0, "b": 0, "Q": 0, "q": 0, "K": 0, "k": 0, "N": 0, "n": 0, "P": 0, "p": 0}

    count = 0
    for x in range(len(start_fen)):
        if start_fen[x] in numberString:
            count += int(start_fen[x])
        elif start_fen[x] == "/":
            pass
        else:        
            fen_letter  = start_fen[x].capitalize()
            class_spelled_out[fen_letter]
            chess_piece_counter[start_fen[x]] += 1
            location = [count//8, count%8]
            
            is_white = False
            if isupper(start_fen[x]):
                piece_name = class_spelled_out[fen_letter] + "White:"
                is_white = True
            else:
                piece_name = class_spelled_out[fen_letter] +"Black:"
            piece_name += (f"{chess_piece_counter[start_fen[x]]}")
        
            chess_pieces[piece_name] = class_dictionary[fen_letter](piece_name, location, is_white, class_single_move[fen_letter])
            count += 1
            
    return chess_pieces
        
chess_pieces = createBoard()

In [419]:
name  = "RookWhite:1"
name2 = "PawnWhite:1"
chess_pieces[name].setPressuredLists()
#chess_pieces[name].setMoves()
    

for piece in chess_pieces:
    chess_pieces[piece].setMoves()




{'RookWhite:1': [0, 0], 'KnightWhite:1': [0, 1], 'BischopWhite:1': [0, 2], 'QueenWhite:1': [0, 3], 'KingWhite:1': [0, 4], 'BischopWhite:2': [0, 5], 'KnightWhite:2': [0, 6], 'RookWhite:2': [0, 7], 'PawnWhite:1': [1, 0], 'PawnWhite:2': [1, 1], 'PawnWhite:3': [1, 2], 'PawnWhite:4': [1, 3], 'PawnWhite:5': [1, 4], 'PawnWhite:6': [1, 5], 'PawnWhite:7': [1, 6], 'PawnWhite:8': [1, 7], 'PawnBlack:1': [6, 0], 'PawnBlack:2': [6, 1], 'PawnBlack:3': [6, 2], 'PawnBlack:4': [6, 3], 'PawnBlack:5': [6, 4], 'PawnBlack:6': [6, 5], 'PawnBlack:7': [6, 6], 'PawnBlack:8': [6, 7], 'RookBlack:1': [7, 0], 'KnightBlack:1': [7, 1], 'BischopBlack:1': [7, 2], 'QueenBlack:1': [7, 3], 'KingBlack:1': [7, 4], 'BischopBlack:2': [7, 5], 'KnightBlack:2': [7, 6], 'RookBlack:2': [7, 7]}

[1, 0]
[[], [2, 2, 2, 2, 2, 2, 2], [2, 2, 2, 2, 2, 2, 2], []]
[[], [2, 2, 2, 2, 2, 2, 2], [0, 0, 0, 0, 0, 1, 2], []]
[[], [2, 2, 2, 2, 2, 2, 2], [0, 0, 0, 0, 0, 1, 2], []]
[7, 0]

{'RookWhite:1': [0, 0], 'KnightWhite:1': [0, 1], 'BischopWhi

In [1]:
x = 5
y = 6
x += 10
print(x)

15
